In [4]:
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as snscollection_recovery_fee                     
import numpy as np
import os
import pandas as pd
import numpy

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

In [5]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [6]:
train_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N


In [7]:
test_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,67991,67991,40000.0,0.0819,814.70,MORTGAGE,140000.0,Not Verified,low_risk,n,...,97.7,0.0,0.0,0.0,527975.0,70914.0,74600.0,99475.0,N,N
1,25429,25429,6000.0,0.1524,208.70,RENT,55000.0,Not Verified,low_risk,n,...,66.7,0.0,0.0,0.0,34628.0,23460.0,5900.0,23628.0,N,N
2,38496,38496,3600.0,0.1695,128.27,RENT,42000.0,Not Verified,low_risk,n,...,100.0,0.0,0.0,0.0,23100.0,19183.0,7300.0,15000.0,N,N
3,19667,19667,20000.0,0.1524,478.33,RENT,100000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,56481.0,43817.0,13800.0,35981.0,N,N
4,37505,37505,3600.0,0.1240,120.27,RENT,50000.0,Not Verified,low_risk,n,...,100.0,25.0,0.0,0.0,45977.0,32448.0,21000.0,24977.0,N,N


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12180 entries, 0 to 12179
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  12180 non-null  int64  
 1   index                       12180 non-null  int64  
 2   loan_amnt                   12180 non-null  float64
 3   int_rate                    12180 non-null  float64
 4   installment                 12180 non-null  float64
 5   home_ownership              12180 non-null  object 
 6   annual_inc                  12180 non-null  float64
 7   verification_status         12180 non-null  object 
 8   loan_status                 12180 non-null  object 
 9   pymnt_plan                  12180 non-null  object 
 10  dti                         12180 non-null  float64
 11  delinq_2yrs                 12180 non-null  float64
 12  inq_last_6mths              12180 non-null  float64
 13  open_acc                    121

In [9]:
train_df.loan_status.value_counts()      

low_risk     6090
high_risk    6090
Name: loan_status, dtype: int64

In [10]:
#https://pythonexamples.org/pandas-dataframe-select-columns-of-numeric-datatype/

result = train_df.select_dtypes(include=['int64',"float64"])
result

,Unnamed: 0,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,57107,57107,13375.0,0.1797,483.34,223000.0,29.99,0.0,0.0,15.0,...,0.0,5.0,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0
1,141451,141451,21000.0,0.1308,478.68,123000.0,11.26,2.0,0.0,16.0,...,0.0,0.0,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0
2,321143,321143,20000.0,0.1240,448.95,197000.0,11.28,0.0,0.0,12.0,...,0.0,1.0,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0
3,11778,11778,3000.0,0.1240,100.22,45000.0,18.08,0.0,0.0,12.0,...,0.0,4.0,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0
4,169382,169382,30000.0,0.1612,1056.49,133000.0,27.77,0.0,2.0,13.0,...,0.0,2.0,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,354912,354912,19975.0,0.2565,801.09,28000.0,28.42,0.0,0.0,15.0,...,0.0,3.0,100.0,16.7,0.0,0.0,50055.0,28192.0,18700.0,19055.0
12176,354944,354944,15000.0,0.1774,540.34,50000.0,23.43,4.0,0.0,16.0,...,2.0,2.0,90.5,11.1,0.0,0.0,70324.0,57025.0,13300.0,54824.0
12177,354973,354973,3600.0,0.1862,131.28,60000.0,28.80,0.0,1.0,14.0,...,0.0,5.0,100.0,0.0,0.0,0.0,83765.0,55156.0,14800.0,53065.0
12178,355002,355002,15000.0,0.0881,475.68,62000.0,11.44,0.0,0.0,5.0,...,0.0,1.0,100.0,0.0,0.0,0.0,189930.0,23748.0,7000.0,32930.0


In [11]:
train_df2 = result.drop(['Unnamed: 0', 'index'], axis=1).reset_index(drop=True)

In [12]:
train_df2.columns

Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_in

In [13]:
cols_num = ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
       'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 
       'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m',
       'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies',
     'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit']
df_num = train_df2.loc[:, cols_num]

scaler = StandardScaler().fit(df_num)
df_scale = pd.DataFrame(scaler.transform(df_num), columns=df_num.columns)
df_scale["loan_status"] = train_df.loan_status 
df_scale

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,loan_status
0,-0.393112,0.736585,-0.087609,0.790279,0.309615,-0.320026,-0.695401,0.376917,-0.351986,0.979339,...,-0.148869,1.326667,0.617581,0.500053,-0.351986,1.965731,1.010651,0.156409,2.188821,low_risk
1,0.351681,-0.191716,-0.103427,0.196561,-0.473994,2.170653,-0.695401,0.539402,-0.351986,-0.354346,...,-0.148869,-1.188713,-1.069844,0.007255,-0.351986,-0.344265,-0.585044,-0.479179,-0.399905,low_risk
2,0.254003,-0.320805,-0.204342,0.635913,-0.473157,-0.320026,-0.695401,-0.110538,-0.351986,-0.039187,...,-0.148869,-0.685637,-0.991097,0.007255,-0.351986,2.230882,0.875447,-0.214679,0.655196,low_risk
3,-1.406519,-0.320805,-1.388060,-0.266539,-0.188665,-0.320026,-0.695401,-0.110538,2.682121,-0.388681,...,-0.148869,0.823591,0.617581,-0.482592,2.682121,-0.815953,-0.706007,-0.321269,-0.783830,low_risk
4,1.230781,0.385387,1.857873,0.255933,0.216736,-0.320026,1.759668,0.051947,-0.351986,2.116088,...,-0.148869,-0.182561,0.617581,0.992851,-0.351986,0.438026,0.790900,2.031592,0.049082,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,0.251561,2.194529,0.990951,-0.367471,0.243930,-0.320026,-0.695401,0.376917,-0.351986,-0.350187,...,-0.148869,0.320515,0.617581,-0.482592,-0.351986,-0.774114,-0.580026,-0.368642,-0.713755,high_risk
12176,-0.234386,0.692922,0.105870,-0.236853,0.035163,4.661331,-0.695401,0.539402,-0.351986,-0.637162,...,4.554681,-0.182561,-0.451122,-0.647841,-0.351986,-0.668756,-0.091207,-0.581820,-0.026850,high_risk
12177,-1.347913,0.859978,-1.282631,-0.177481,0.259828,-0.320026,0.532133,0.214432,-0.351986,-0.256166,...,-0.148869,1.326667,0.617581,-0.975390,-0.351986,-0.598889,-0.122893,-0.522604,-0.060630,high_risk
12178,-0.234386,-1.002318,-0.113610,-0.165607,-0.466463,-0.320026,-0.695401,-1.247934,-0.351986,-0.771491,...,-0.148869,-0.685637,0.617581,-0.975390,-0.351986,-0.047042,-0.655367,-0.830529,-0.447301,high_risk


In [14]:
#https://towardsdatascience.com/guide-to-encoding-categorical-features-using-scikit-learn-for-machine-learning-5048997a5c79

#df_scale2 = df_scale["Loan Status"].map({"low_risk":0,"high_risk":1})
#df_scale2

#df_scale2.shape
df_scale["loan_status"] = [1 if x == "high_risk" else 0 for x in df_scale.loan_status]
df_scale.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,loan_status
0,-0.393112,0.736585,-0.087609,0.790279,0.309615,-0.320026,-0.695401,0.376917,-0.351986,0.979339,...,-0.148869,1.326667,0.617581,0.500053,-0.351986,1.965731,1.010651,0.156409,2.188821,0
1,0.351681,-0.191716,-0.103427,0.196561,-0.473994,2.170653,-0.695401,0.539402,-0.351986,-0.354346,...,-0.148869,-1.188713,-1.069844,0.007255,-0.351986,-0.344265,-0.585044,-0.479179,-0.399905,0
2,0.254003,-0.320805,-0.204342,0.635913,-0.473157,-0.320026,-0.695401,-0.110538,-0.351986,-0.039187,...,-0.148869,-0.685637,-0.991097,0.007255,-0.351986,2.230882,0.875447,-0.214679,0.655196,0
3,-1.406519,-0.320805,-1.388060,-0.266539,-0.188665,-0.320026,-0.695401,-0.110538,2.682121,-0.388681,...,-0.148869,0.823591,0.617581,-0.482592,2.682121,-0.815953,-0.706007,-0.321269,-0.783830,0
4,1.230781,0.385387,1.857873,0.255933,0.216736,-0.320026,1.759668,0.051947,-0.351986,2.116088,...,-0.148869,-0.182561,0.617581,0.992851,-0.351986,0.438026,0.790900,2.031592,0.049082,0


In [15]:
result2 = train_df.select_dtypes(include=['object'])
result2

,home_ownership,verification_status,loan_status,pymnt_plan,initial_list_status,application_type,hardship_flag,debt_settlement_flag
0,MORTGAGE,Not Verified,low_risk,n,w,Individual,N,N
1,MORTGAGE,Source Verified,low_risk,n,w,Individual,N,N
2,MORTGAGE,Source Verified,low_risk,n,w,Individual,N,N
3,RENT,Not Verified,low_risk,n,w,Individual,N,N
4,MORTGAGE,Source Verified,low_risk,n,w,Individual,N,N
...,...,...,...,...,...,...,...,...
12175,RENT,Not Verified,high_risk,n,w,Individual,N,N
12176,RENT,Verified,high_risk,n,w,Individual,N,N
12177,RENT,Not Verified,high_risk,n,w,Individual,N,N
12178,MORTGAGE,Source Verified,high_risk,n,w,Joint App,N,N


In [16]:
result2.columns

Index(['home_ownership', 'verification_status', 'loan_status', 'pymnt_plan',
       'initial_list_status', 'application_type', 'hardship_flag',
       'debt_settlement_flag'],
      dtype='object')

In [17]:
result3=pd.get_dummies(result2, dummy_na=True)
result3

,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,home_ownership_nan,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,verification_status_nan,loan_status_high_risk,...,initial_list_status_nan,application_type_Individual,application_type_Joint App,application_type_nan,hardship_flag_N,hardship_flag_Y,hardship_flag_nan,debt_settlement_flag_N,debt_settlement_flag_Y,debt_settlement_flag_nan
0,0,1,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,1,0,0
2,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,1,0,0
3,0,0,0,1,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
4,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,0,0,0,1,0,1,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
12176,0,0,0,1,0,0,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0
12177,0,0,0,1,0,1,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
12178,0,1,0,0,0,0,1,0,0,1,...,0,0,1,0,1,0,0,1,0,0


In [18]:
result4 = pd.concat([df_scale, result3], axis=1)
result5=result4.dropna(how='all')


In [19]:
for col in result4.columns:
    print(col)

loan_amnt
int_rate
installment
annual_inc
dti
delinq_2yrs
inq_last_6mths
open_acc
pub_rec
revol_bal
total_acc
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
last_pymnt_amnt
collections_12_mths_ex_med
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
bc_util
chargeoff_within_12_mths
delinq_amnt
mo_sin_old_il_acct
mo_sin_old_rev_tl_op
mo_sin_rcnt_rev_tl_op
mo_sin_rcnt_tl
mort_acc
mths_since_recent_bc
mths_since_recent_inq
num_accts_ever_120_pd
num_actv_bc_tl
num_actv_rev_tl
num_bc_sats
num_bc_tl
num_il_tl
num_op_rev_tl
num_rev_accts
num_rev_tl_bal_gt_0
num_sats
num_tl_30dpd
num_tl_90g_dpd_24m
num_tl_op_past_12m
pct_tl_nvr_dlq
percent_bc_gt_75
pub_rec_bankruptcies
tot_hi_cred_lim
total_bal_ex_mort
total_bc_limit
total_il

In [20]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4702 entries, 0 to 4701
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  4702 non-null   int64  
 1   index                       4702 non-null   int64  
 2   loan_amnt                   4702 non-null   float64
 3   int_rate                    4702 non-null   float64
 4   installment                 4702 non-null   float64
 5   home_ownership              4702 non-null   object 
 6   annual_inc                  4702 non-null   float64
 7   verification_status         4702 non-null   object 
 8   loan_status                 4702 non-null   object 
 9   pymnt_plan                  4702 non-null   object 
 10  dti                         4702 non-null   float64
 11  delinq_2yrs                 4702 non-null   float64
 12  inq_last_6mths              4702 non-null   float64
 13  open_acc                    4702 

In [21]:
cols_num2 = ["loan_amnt",
"int_rate",
"installment",
"annual_inc",
"dti",
"delinq_2yrs",
"inq_last_6mths",
"open_acc",
"pub_rec",
"revol_bal",
"total_acc",
"out_prncp",
"out_prncp_inv",
"total_pymnt",
"total_pymnt_inv",
"total_rec_prncp",
"total_rec_int",
"total_rec_late_fee",
"recoveries",
"collection_recovery_fee",
"last_pymnt_amnt",
"collections_12_mths_ex_med",
"policy_code",
"acc_now_delinq",
"tot_coll_amt",
"tot_cur_bal",
"open_acc_6m",
"open_act_il",
"open_il_12m",
"open_il_24m",
"mths_since_rcnt_il",
"total_bal_il",
"il_util",
"open_rv_12m",
"open_rv_24m",
"max_bal_bc",
"all_util",
"total_rev_hi_lim",
"inq_fi",
"total_cu_tl",
"inq_last_12m",
"acc_open_past_24mths",
"avg_cur_bal",
"bc_open_to_buy",
"bc_util",
"chargeoff_within_12_mths",
"delinq_amnt",
"mo_sin_old_il_acct",
"mo_sin_old_rev_tl_op",
"mo_sin_rcnt_rev_tl_op",
"mo_sin_rcnt_tl",
"mort_acc",
"mths_since_recent_bc",
"mths_since_recent_inq",
"num_accts_ever_120_pd",
"num_actv_bc_tl",
"num_actv_rev_tl",
"num_bc_sats",
"num_bc_tl",
"num_il_tl",
"num_op_rev_tl",
"num_rev_accts",
"num_rev_tl_bal_gt_0",
"num_sats",
"num_tl_120dpd_2m",
"num_tl_30dpd",
"num_tl_90g_dpd_24m",
"num_tl_op_past_12m",
"pct_tl_nvr_dlq",
"percent_bc_gt_75",
"pub_rec_bankruptcies",
"tax_liens",
"tot_hi_cred_lim",
"total_bal_ex_mort",
"total_bc_limit",
"total_il_high_credit_limit",
"loan_status",
'home_ownership', 'verification_status', 'loan_status', 'pymnt_plan',
'initial_list_status', 'application_type', 'hardship_flag',
'debt_settlement_flag']

In [22]:
df_num2 = test_df.loc[:, cols_num2]
df_num2

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_il_high_credit_limit,loan_status,home_ownership,verification_status,loan_status,pymnt_plan,initial_list_status,application_type,hardship_flag,debt_settlement_flag
0,40000.0,0.0819,814.70,140000.0,19.75,0.0,1.0,18.0,0.0,9471.0,...,99475.0,low_risk,MORTGAGE,Not Verified,low_risk,n,w,Individual,N,N
1,6000.0,0.1524,208.70,55000.0,11.52,2.0,0.0,8.0,0.0,1280.0,...,23628.0,low_risk,RENT,Not Verified,low_risk,n,w,Individual,N,N
2,3600.0,0.1695,128.27,42000.0,6.74,0.0,0.0,6.0,0.0,4757.0,...,15000.0,low_risk,RENT,Not Verified,low_risk,n,w,Individual,N,N
3,20000.0,0.1524,478.33,100000.0,12.13,0.0,2.0,7.0,0.0,12731.0,...,35981.0,low_risk,RENT,Not Verified,low_risk,n,w,Individual,N,N
4,3600.0,0.1240,120.27,50000.0,16.08,0.0,3.0,6.0,0.0,10413.0,...,24977.0,low_risk,RENT,Not Verified,low_risk,n,w,Individual,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,30000.0,0.1240,673.42,140480.0,15.74,0.0,0.0,20.0,0.0,23215.0,...,107388.0,high_risk,RENT,Source Verified,high_risk,n,f,Individual,N,N
4698,24000.0,0.0756,747.22,50000.0,26.81,0.0,0.0,9.0,0.0,459.0,...,30775.0,high_risk,RENT,Not Verified,high_risk,n,w,Individual,N,N
4699,10000.0,0.2305,387.36,33000.0,38.51,0.0,2.0,7.0,0.0,6342.0,...,29550.0,high_risk,RENT,Verified,high_risk,n,f,Individual,N,N
4700,8000.0,0.1862,205.86,38000.0,16.36,0.0,1.0,8.0,1.0,11636.0,...,9657.0,high_risk,RENT,Source Verified,high_risk,n,w,Individual,N,N


In [23]:
result5 = df_num2.select_dtypes(include=['int64',"float64"])
result5.columns

Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_in

In [24]:
cols_num5 = ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 
       'last_pymnt_amnt', 'collections_12_mths_ex_med',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
       'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats',
       'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m',
       'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies',
        'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit']
df_num10 = result5.loc[:, cols_num5]

scaler = StandardScaler().fit(df_num10)
df_scale10 = pd.DataFrame(scaler.transform(df_num10), columns=df_num10.columns)
df_scale10["loan_status"] = test_df.loan_status 
df_scale10

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,loan_status
0,2.273948,-1.132822,1.061318,0.917459,-0.152842,-0.323913,0.331114,0.875022,-0.334196,-0.404119,...,-0.164647,-0.260036,0.299444,-0.976752,-0.334196,1.745527,0.146528,1.591187,0.887063,low_risk
1,-1.085188,0.084801,-1.005468,-0.547639,-0.541770,2.829374,-0.778578,-0.779056,-0.334196,-0.768384,...,-0.164647,-0.748331,-3.604666,-0.976752,-0.334196,-0.843948,-0.705592,-0.892419,-0.646997,low_risk
2,-1.322303,0.380140,-1.279777,-0.771713,-0.767660,-0.323913,-0.778578,-1.109872,-0.334196,-0.613757,...,-0.164647,-0.748331,0.589104,-0.976752,-0.334196,-0.904456,-0.782393,-0.841807,-0.821505,low_risk
3,0.297986,0.084801,-0.085885,0.228001,-0.512943,-0.323913,1.440806,-0.944464,-0.334196,-0.259142,...,-0.164647,0.716553,0.589104,0.530853,-0.334196,-0.729246,-0.340046,-0.606823,-0.397149,low_risk
4,-1.322303,-0.405702,-1.307061,-0.633822,-0.326276,-0.323913,2.550498,-1.109872,-0.334196,-0.362227,...,-0.164647,0.228259,0.589104,-0.222950,-0.334196,-0.784379,-0.544196,-0.346532,-0.619713,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,1.285967,-0.405702,0.579477,0.925732,-0.342344,-0.323913,-0.778578,1.205838,-0.334196,0.207096,...,-0.164647,-0.260036,0.589104,-0.114402,-0.334196,-0.187534,0.864062,0.644018,1.047109,high_risk
4698,0.693178,-1.241631,0.831175,-0.633822,0.180795,-0.323913,-0.778578,-0.613648,-0.334196,-0.804895,...,-0.164647,-0.260036,0.589104,-0.976752,-0.334196,-0.698310,-0.786972,-0.624898,-0.502444,high_risk
4699,-0.689995,1.433686,-0.396141,-0.926841,0.733706,-0.323913,1.440806,-0.944464,-0.334196,-0.543269,...,-0.164647,1.204848,0.589104,-0.976752,-0.334196,-0.798693,-0.533889,-0.798426,-0.527220,high_risk
4700,-0.887591,0.668570,-1.015154,-0.840659,-0.313044,-0.323913,0.331114,-0.779056,2.899116,-0.307838,...,-0.164647,0.228259,-0.040591,-0.976752,2.899116,-0.861117,-0.774995,-1.051486,-0.929571,high_risk


In [25]:
#df_scale20 = df_scale["Loan Status"].map({"low_risk":0,"high_risk":1})
#df_scale20

df_scale10["loan_status"] = [1 if x == "high_risk" else 0 for x in df_scale10["loan_status"]]
df_scale10.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,loan_status
0,2.273948,-1.132822,1.061318,0.917459,-0.152842,-0.323913,0.331114,0.875022,-0.334196,-0.404119,...,-0.164647,-0.260036,0.299444,-0.976752,-0.334196,1.745527,0.146528,1.591187,0.887063,0
1,-1.085188,0.084801,-1.005468,-0.547639,-0.541770,2.829374,-0.778578,-0.779056,-0.334196,-0.768384,...,-0.164647,-0.748331,-3.604666,-0.976752,-0.334196,-0.843948,-0.705592,-0.892419,-0.646997,0
2,-1.322303,0.380140,-1.279777,-0.771713,-0.767660,-0.323913,-0.778578,-1.109872,-0.334196,-0.613757,...,-0.164647,-0.748331,0.589104,-0.976752,-0.334196,-0.904456,-0.782393,-0.841807,-0.821505,0
3,0.297986,0.084801,-0.085885,0.228001,-0.512943,-0.323913,1.440806,-0.944464,-0.334196,-0.259142,...,-0.164647,0.716553,0.589104,0.530853,-0.334196,-0.729246,-0.340046,-0.606823,-0.397149,0
4,-1.322303,-0.405702,-1.307061,-0.633822,-0.326276,-0.323913,2.550498,-1.109872,-0.334196,-0.362227,...,-0.164647,0.228259,0.589104,-0.222950,-0.334196,-0.784379,-0.544196,-0.346532,-0.619713,0


In [26]:
result25 = test_df.select_dtypes(include=['object'])
result25

,home_ownership,verification_status,loan_status,pymnt_plan,initial_list_status,application_type,hardship_flag,debt_settlement_flag
0,MORTGAGE,Not Verified,low_risk,n,w,Individual,N,N
1,RENT,Not Verified,low_risk,n,w,Individual,N,N
2,RENT,Not Verified,low_risk,n,w,Individual,N,N
3,RENT,Not Verified,low_risk,n,w,Individual,N,N
4,RENT,Not Verified,low_risk,n,w,Individual,N,N
...,...,...,...,...,...,...,...,...
4697,RENT,Source Verified,high_risk,n,f,Individual,N,N
4698,RENT,Not Verified,high_risk,n,w,Individual,N,N
4699,RENT,Verified,high_risk,n,f,Individual,N,N
4700,RENT,Source Verified,high_risk,n,w,Individual,N,N


In [27]:
result30=pd.get_dummies(result25, dummy_na=True)
result30

,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,home_ownership_nan,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,verification_status_nan,loan_status_high_risk,...,initial_list_status_w,initial_list_status_nan,application_type_Individual,application_type_Joint App,application_type_nan,hardship_flag_N,hardship_flag_Y,hardship_flag_nan,debt_settlement_flag_N,debt_settlement_flag_nan
0,0,1,0,0,0,1,0,0,0,0,...,1,0,1,0,0,1,0,0,1,0
1,0,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,1,0,0,1,0
2,0,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,1,0,0,1,0
3,0,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,1,0,0,1,0
4,0,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,0,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,1,0
4698,0,0,0,1,0,1,0,0,0,1,...,1,0,1,0,0,1,0,0,1,0
4699,0,0,0,1,0,0,0,1,0,1,...,0,0,1,0,0,1,0,0,1,0
4700,0,0,0,1,0,0,1,0,0,1,...,1,0,1,0,0,1,0,0,1,0


In [28]:
result40 = pd.concat([df_scale, result30], axis=1)
result41=result40.dropna(how='all')

In [29]:
#I believe that the logistic Regression will perform better than the Random Forest

In [33]:
df_scale10

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,loan_status
0,2.273948,-1.132822,1.061318,0.917459,-0.152842,-0.323913,0.331114,0.875022,-0.334196,-0.404119,...,-0.164647,-0.260036,0.299444,-0.976752,-0.334196,1.745527,0.146528,1.591187,0.887063,0
1,-1.085188,0.084801,-1.005468,-0.547639,-0.541770,2.829374,-0.778578,-0.779056,-0.334196,-0.768384,...,-0.164647,-0.748331,-3.604666,-0.976752,-0.334196,-0.843948,-0.705592,-0.892419,-0.646997,0
2,-1.322303,0.380140,-1.279777,-0.771713,-0.767660,-0.323913,-0.778578,-1.109872,-0.334196,-0.613757,...,-0.164647,-0.748331,0.589104,-0.976752,-0.334196,-0.904456,-0.782393,-0.841807,-0.821505,0
3,0.297986,0.084801,-0.085885,0.228001,-0.512943,-0.323913,1.440806,-0.944464,-0.334196,-0.259142,...,-0.164647,0.716553,0.589104,0.530853,-0.334196,-0.729246,-0.340046,-0.606823,-0.397149,0
4,-1.322303,-0.405702,-1.307061,-0.633822,-0.326276,-0.323913,2.550498,-1.109872,-0.334196,-0.362227,...,-0.164647,0.228259,0.589104,-0.222950,-0.334196,-0.784379,-0.544196,-0.346532,-0.619713,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,1.285967,-0.405702,0.579477,0.925732,-0.342344,-0.323913,-0.778578,1.205838,-0.334196,0.207096,...,-0.164647,-0.260036,0.589104,-0.114402,-0.334196,-0.187534,0.864062,0.644018,1.047109,1
4698,0.693178,-1.241631,0.831175,-0.633822,0.180795,-0.323913,-0.778578,-0.613648,-0.334196,-0.804895,...,-0.164647,-0.260036,0.589104,-0.976752,-0.334196,-0.698310,-0.786972,-0.624898,-0.502444,1
4699,-0.689995,1.433686,-0.396141,-0.926841,0.733706,-0.323913,1.440806,-0.944464,-0.334196,-0.543269,...,-0.164647,1.204848,0.589104,-0.976752,-0.334196,-0.798693,-0.533889,-0.798426,-0.527220,1
4700,-0.887591,0.668570,-1.015154,-0.840659,-0.313044,-0.323913,0.331114,-0.779056,2.899116,-0.307838,...,-0.164647,0.228259,-0.040591,-0.976752,2.899116,-0.861117,-0.774995,-1.051486,-0.929571,1


In [35]:
X_train = result5.values
y_train = df_scale["loan_status"].values

X_test = result41.values
y_test = df_scale10["loan_status"].values

In [37]:
X_train.shape

(4702, 76)

In [38]:
y_train.shape

(12180,)

In [36]:
# Initialize the model
model1 = LogisticRegression()

# FIT THE MODEL
model1.fit(X_train, y_train)

# EVALUATE the MODEL
train_preds = model1.predict(X_train)
test_preds = model1.predict(X_test)

# PRINT the METRICS
print("TRAINING SET")

class_report_train = classification_report(y_train, train_preds)
conf_matrix_train = confusion_matrix(y_train, train_preds)

print(class_report_train)
print(conf_matrix_train)

print()

# PRINT the METRICS
print("Testing SET")

class_report_test = classification_report(y_test, test_preds)
conf_matrix_test = confusion_matrix(y_test, test_preds)

print(class_report_test)
print(conf_matrix_test)

ValueError: Found input variables with inconsistent numbers of samples: [4702, 12180]